In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

We read the data

In [7]:
path_x = Path.cwd().parent.joinpath("data", "raw_x.xls")
print(f"This is the path of raw x: {path_x}")
raw_x = pd.read_excel(path_x, skiprows=1).iloc[2:,:]
raw_x.head()

This is the path of raw x: c:\Documentos\proyectos\xm4ecast\data\raw_x.xls
WARNING *** file size (87872) not 512 + multiple of sector size (512)


,Unnamed: 0,1998,Unnamed: 2,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022*
2,Definitivo,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.036826e+06,5.352381e+06,4.967463e+06,5.075730e+06,5.347056e+06,5.416194e+06,5.235324e+06,5.118868e+06,5.966781e+06,1.615367e+06
3,Perfeccionamiento Activo,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.170279e+05,1.433158e+05,1.168891e+05,1.311091e+05,1.571880e+05,1.443899e+05,1.544929e+05,1.388538e+05,2.491643e+05,6.777765e+04
4,Zona Franca,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.345952e+06,3.671282e+06,4.154194e+06,4.715010e+06,5.109536e+06,5.668595e+06,5.974035e+06,6.371646e+06,8.270916e+06,2.191665e+06
5,Grand Total,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.599806e+06,9.166979e+06,9.238546e+06,9.921850e+06,1.061378e+07,1.122918e+07,1.136385e+07,1.162937e+07,1.448686e+07,3.874809e+06


In [30]:
raw_x = raw_x.rename(columns={raw_x.columns[0]: "reg"})
data_x = (
    raw_x.melt(
        id_vars="reg",
        value_vars=[e for e in raw_x.columns if e != "reg"],
        value_name="value",
        var_name="year",
    )
    .query("value == value")
    .assign(
        year=lambda x: x["year"].str.slice(0, 4),
        value=lambda x: pd.to_numeric(x["value"]),
        reg=lambda x: x["reg"].replace(
            {
                "Definitivo": "normal",
                "Perfeccionamiento Activo": "especial",
                "Zona Franca": "especial",
            }
        ),
    )
    .query("reg != 'Grand Total'")
    .groupby(["reg", "year"])
    .agg({"value" : np.sum})
    .reset_index()
    .assign(type = "x")
)
data_x.tail()


,reg,year,value,type
45,normal,2018,5.416194e+06,x
46,normal,2019,5.235324e+06,x
47,normal,2020,5.118868e+06,x
48,normal,2021,5.966781e+06,x
49,normal,2022,1.615367e+06,x


Here I saw that we can get the data in monthly frecuency. So, the Central Bank Method si prefered.